In [6]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('../food_inflation_analysis.db')

# Get a list of all tables in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Create an empty DataFrame to store the combined data
combined_data = pd.DataFrame()



In [8]:
tables

[('interest_rate',),
 ('test_data',),
 ('OCED_USA_FOOD_INFLATION',),
 ('food_production',)]

In [9]:
# Iterate over each table
for table in tables:
    table_name = table[0]
    
    # Read the table data into a DataFrame
    table_data = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    print(table_data)
    # # Check if the table has a different time unit
    # if 'year' not in table_data.columns:
    #     # Convert the data to yearly by repeating the values for each year
    #     table_data['year'] = pd.DatetimeIndex(table_data['date']).year
    #     table_data = table_data.groupby('year').ffill()
    
    # Append the table data to the combined DataFrame
    combined_data = combined_data.append(table_data, ignore_index=True)

    
# Close the database connection
conn.close()

# Print the combined data
print(combined_data)

     FEDFUNDS
0        0.80
1        1.22
2        1.07
3        0.85
4        0.83
..        ...
827      5.08
828      5.12
829      5.33
830      5.33
831      5.33

[832 rows x 1 columns]
                  Date  Inflation
0  2023-01-01 00:00:00  11.357280
1  2023-02-01 00:00:00  10.149780
2  2023-03-01 00:00:00   8.325572
3  2023-04-01 00:00:00   7.009775
4  2023-05-01 00:00:00   5.647190
5  2023-06-01 00:00:00   4.581234
6  2023-07-01 00:00:00   3.485868
7  2023-08-01 00:00:00   2.893452
8  2023-09-01 00:00:00   2.376901
                    Date  Inflation
0    1956-01-01 00:00:00  -1.700680
1    1956-02-01 00:00:00  -2.372881
2    1956-03-01 00:00:00  -2.033898
3    1956-04-01 00:00:00  -1.689189
4    1956-05-01 00:00:00  -0.337838
..                   ...        ...
799  2022-08-01 00:00:00  13.533210
800  2022-09-01 00:00:00  12.927050
801  2022-10-01 00:00:00  12.439140
802  2022-11-01 00:00:00  11.956870
803  2022-12-01 00:00:00  11.882750

[804 rows x 2 columns]
   Country C

C:\AppData\AppData\Local\Temp\ipykernel_9828\676636180.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(table_data, ignore_index=True)
C:\AppData\AppData\Local\Temp\ipykernel_9828\676636180.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(table_data, ignore_index=True)
C:\AppData\AppData\Local\Temp\ipykernel_9828\676636180.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(table_data, ignore_index=True)
C:\AppData\AppData\Local\Temp\ipykernel_9828\676636180.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined

In [10]:
combined_data

,FEDFUNDS,Date,Inflation,Country Code,Year,Cereal_Production,Food_Production,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9
0,0.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701,NaN,NaN,NaN,USA,2017.0,440271775.0,101.96,104.86,97.01,98.13,95.81,90.90,91.37,92.23,93.04,91.21
1702,NaN,NaN,NaN,USA,2018.0,439707894.8,103.20,101.96,104.86,97.01,98.13,95.81,90.90,91.37,92.23,93.04
1703,NaN,NaN,NaN,USA,2019.0,420869946.6,99.65,103.20,101.96,104.86,97.01,98.13,95.81,90.90,91.37,92.23
1704,NaN,NaN,NaN,USA,2020.0,433256752.7,104.05,99.65,103.20,101.96,104.86,97.01,98.13,95.81,90.90,91.37
